In [1]:
import pandas as pd
import h2o
from h2o.estimators import H2OXGBoostEstimator
from h2o.estimators.deeplearning import H2ODeepLearningEstimator
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator
import optuna

seed = 1

In [2]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.21" 2023-10-17; OpenJDK Runtime Environment (build 11.0.21+9-post-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.21+9-post-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp_n7h4cyd
  JVM stdout: /tmp/tmp_n7h4cyd/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp_n7h4cyd/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,1 month and 8 days
H2O_cluster_name:,H2O_from_python_unknownUser_7htgnk
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,7.250 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [4]:
#get data
train_clean = pd.read_csv('../prediction-task/train-clean.csv')
test_clean = pd.read_csv('../prediction-task/test-clean.csv')

In [6]:
#convert pandas df to h2o df
train_clean_h2o = h2o.H2OFrame(train_clean)

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [8]:
#ensure categorical columns are understood as categorical by h2o models
train_clean_h2o['MODE'] = train_clean_h2o['MODE'].asfactor()
train_clean_h2o['POWER'] = train_clean_h2o['POWER'].asfactor()

In [10]:
#split data into train and val sets
splits = train_clean_h2o.split_frame(ratios=[0.9], seed=1) 
train = splits[0]
val = splits[1]

In [12]:
#get features and target for modelling
y = 'OUTPUT'
x = list(train.columns)
x.remove(y)

## tune multiple deep learning models with different hyperparameters

In [16]:
dl_models = []

def objective(trial):
    #params to tune
    num_layers = trial.suggest_int('num_layers', 1, 5)
    hidden_layer_size = trial.suggest_int('hidden_layer_size', 100, 300, step=50)

    params = {
        'hidden': [hidden_layer_size]*num_layers,
        'activation': trial.suggest_categorical('activation', ['rectifier', 'rectifierwithdropout', 'tanh', 'tanh_with_dropout', 'maxout', 'maxout_with_dropout']),
        'epochs': trial.suggest_int('epochs', 5, 50, step=5),
    }

    #train model
    model = H2ODeepLearningEstimator(**params,
                                    standardize=True,
                                    categorical_encoding='auto',
                                    nfolds=5,
                                    keep_cross_validation_predictions=True,  #need for stacked ensembling later
                                    seed=seed)
    model.train(x=x, y=y, training_frame=train)
    
    #store model
    dl_models.append(model)

    #get cv rmse
    cv_metrics_df = model.cross_validation_metrics_summary().as_data_frame()
    cv_rmse_index = cv_metrics_df[cv_metrics_df[''] == 'rmse'].index
    cv_rmse = cv_metrics_df['mean'].iloc[cv_rmse_index]
    return cv_rmse


In [17]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

[I 2023-12-17 05:00:24,791] A new study created in memory with name: no-name-ecca3309-e496-456a-b47e-3b4b87ce8d56


deeplearning Model Build progress: |████████████████████████████████████████████

[I 2023-12-17 05:00:30,906] Trial 0 finished with value: 1.6633766 and parameters: {'num_layers': 1, 'hidden_layer_size': 300, 'activation': 'tanh_with_dropout', 'epochs': 30}. Best is trial 0 with value: 1.6633766.


█| (done) 100%
deeplearning Model Build progress: |█████████████████████████████████████████████

[I 2023-12-17 05:05:20,643] Trial 1 finished with value: 0.4778851 and parameters: {'num_layers': 3, 'hidden_layer_size': 300, 'activation': 'tanh', 'epochs': 40}. Best is trial 1 with value: 0.4778851.


...

█| (done) 100%
deeplearning Model Build progress: |████████████████████████████████████████████

[I 2023-12-17 05:26:40,232] Trial 18 finished with value: 0.49151322 and parameters: {'num_layers': 3, 'hidden_layer_size': 250, 'activation': 'tanh', 'epochs': 35}. Best is trial 1 with value: 0.4778851.


█| (done) 100%
deeplearning Model Build progress: |████████████████████████████████████████████

[I 2023-12-17 05:28:30,272] Trial 19 finished with value: 0.5302075 and parameters: {'num_layers': 4, 'hidden_layer_size': 200, 'activation': 'tanh', 'epochs': 25}. Best is trial 1 with value: 0.4778851.


█| (done) 100%


In [18]:
len(dl_models)

20

## tune multiple xgboost models with different hyperparameters

In [13]:
xgboost_models = []

def objective(trial):
    #common params between xgboost and lightgbm
    params = {
        'ntrees': trial.suggest_int('ntrees', 50, 5000),
        'max_depth': trial.suggest_int('max_depth', 1, 9),
        'min_rows': trial.suggest_int('min_rows', 1, 5),
        'sample_rate': trial.suggest_float('sample_rate', 0.8, 1.0),
        'col_sample_rate': trial.suggest_float('col_sample_rate', 0.2, 1.0),
        'col_sample_rate_per_tree': trial.suggest_float('col_sample_rate_per_tree', 0.5, 1.0)
    }
    
    grow_policy = trial.suggest_categorical('grow_policy', ['depthwise', 'lossguide'])
    
    #add lightgbm-only params
    if grow_policy == 'lossguide':  #emulates lightgbm
        tree_method = 'hist'  #must be hist when using lossguide
        params['max_bins'] = trial.suggest_int('max_bins', 20, 256)
        params['max_leaves'] = trial.suggest_int('max_leaves', 31, 1024)
        
    #add xgboost-only params
    else:
        tree_method = 'auto'
        params['booster'] = trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart'])
        params['reg_alpha'] = trial.suggest_float('reg_alpha', 0.001, 1)
        params['reg_lambda'] = trial.suggest_float('reg_lambda', 0.001, 1)
        params['min_split_improvement'] = trial.suggest_float('min_split_improvement', 1e-10, 1e-3, log=True)
    
    #add grow_policy and tree_method into params
    params['grow_policy'] = grow_policy
    params['tree_method'] = tree_method

    #train model
    model = H2OXGBoostEstimator(**params,
                                learn_rate=0.1,
                                categorical_encoding='auto',
                                nfolds=5,
                                keep_cross_validation_predictions=True,  #need for stacked ensembling later
                                seed=seed) 
    model.train(x=x, y=y, training_frame=train)

    #store model
    xgboost_models.append(model)

    #get cv rmse
    cv_metrics_df = model.cross_validation_metrics_summary().as_data_frame()
    cv_rmse_index = cv_metrics_df[cv_metrics_df[''] == 'rmse'].index
    cv_rmse = cv_metrics_df['mean'].iloc[cv_rmse_index]
    return cv_rmse


In [14]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

[I 2023-12-17 04:33:35,373] A new study created in memory with name: no-name-c695d077-f1f0-4b38-92c3-94fea2beaf6e


xgboost Model Build progress: |█████████████████████████████████████████████████

/opt/conda/lib/python3.10/site-packages/optuna/study/_tell.py:66: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(v)
/opt/conda/lib/python3.10/site-packages/optuna/study/_tell.py:70: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  if math.isnan(v):
/opt/conda/lib/python3.10/site-packages/optuna/study/_tell.py:168: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  values = [float(value) for value in values]
[I 2023-12-17 04:34:11,053] Trial 0 finished with value: 0.91742015 and parameters: {'ntrees': 2450, 'max_depth': 4, 'min_rows': 1, 'sample_rate': 0.8188631129384818, 'col_sample_rate': 0.7707111398310464, 'col_sample_rate_per_tree': 0.9761972625178172, 'grow_policy': 'depthwise', 'booster': 'g

█| (done) 100%
xgboost Model Build progress: |█████████████████████████████████████████████████

[I 2023-12-17 04:34:33,960] Trial 1 finished with value: 0.8052408 and parameters: {'ntrees': 1513, 'max_depth': 9, 'min_rows': 1, 'sample_rate': 0.8987283320043595, 'col_sample_rate': 0.9393631519305861, 'col_sample_rate_per_tree': 0.9502997317497792, 'grow_policy': 'lossguide', 'max_bins': 158, 'max_leaves': 1012}. Best is trial 1 with value: 0.8052408.


...

█| (done) 100%
xgboost Model Build progress: |█████████████████████████████████████████████████

[I 2023-12-17 04:51:14,266] Trial 18 finished with value: 0.7871454 and parameters: {'ntrees': 741, 'max_depth': 6, 'min_rows': 2, 'sample_rate': 0.9618181597901027, 'col_sample_rate': 0.8380692178167941, 'col_sample_rate_per_tree': 0.8483749309839395, 'grow_policy': 'depthwise', 'booster': 'dart', 'reg_alpha': 0.964316366336372, 'reg_lambda': 0.09919986407233439, 'min_split_improvement': 0.00012696661324644446}. Best is trial 18 with value: 0.7871454.


█| (done) 100%
xgboost Model Build progress: |█████████████████████████████████████████████████

[I 2023-12-17 05:00:24,751] Trial 19 finished with value: 0.863278 and parameters: {'ntrees': 704, 'max_depth': 6, 'min_rows': 2, 'sample_rate': 0.976209037093354, 'col_sample_rate': 0.6871850363941621, 'col_sample_rate_per_tree': 0.8339593470448108, 'grow_policy': 'depthwise', 'booster': 'dart', 'reg_alpha': 0.9651393268304185, 'reg_lambda': 0.15124209937660504, 'min_split_improvement': 0.0005083777188725635}. Best is trial 18 with value: 0.7871454.


█| (done) 100%


In [15]:
len(xgboost_models)

20

## build stacked ensembler using deep learning and xgboost models
with GLM chosen as meta-model

In [19]:
base_models = dl_models + xgboost_models

In [20]:
len(base_models)

40

In [21]:
def objective(trial):   
    meta_model_params = {
        'alpha': trial.suggest_float('alpha', 0, 1),  #regularization distribution between L1 and L2
        'family': trial.suggest_categorical('family', ['gaussian', 'tweedie']),  
        'lambda': trial.suggest_float('lambda', 1e-6, 10.0, log=True),
        'standardize': trial.suggest_categorical('standardize', [True, False]),
        'non_negative': True  #predictions of each base model cannot be subtracted from one another
    }

    ensemble = H2OStackedEnsembleEstimator(metalearner_algorithm='glm',
                                             metalearner_params=meta_model_params,
                                             metalearner_nfolds=5,
                                             base_models=base_models,  
                                             seed=seed)

    ensemble.train(x=x, y=y, training_frame=train)
    
    #get cv rmse
    cv_metrics_df = ensemble.cross_validation_metrics_summary().as_data_frame()
    cv_rmse_index = cv_metrics_df[cv_metrics_df[''] == 'rmse'].index
    cv_rmse = cv_metrics_df['mean'].iloc[cv_rmse_index]
    return cv_rmse
    

In [22]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

[I 2023-12-17 05:28:30,330] A new study created in memory with name: no-name-0de7a2c8-3133-42ce-a22a-a55c81bb429c


stackedensemble Model Build progress: |

[I 2023-12-17 05:28:31,683] Trial 0 finished with value: 0.4435823 and parameters: {'alpha': 0.2883819606738718, 'family': 'tweedie', 'lambda': 1.0078116253602135e-05, 'standardize': True}. Best is trial 0 with value: 0.4435823.


██████████████████████████████████████████| (done) 100%
stackedensemble Model Build progress: |

[I 2023-12-17 05:28:32,784] Trial 1 finished with value: 0.44359657 and parameters: {'alpha': 0.07344939994794075, 'family': 'tweedie', 'lambda': 2.7033032441268117e-05, 'standardize': False}. Best is trial 0 with value: 0.4435823.


...

██████████████████████████████████████████| (done) 100%
stackedensemble Model Build progress: |

[I 2023-12-17 05:28:47,852] Trial 18 finished with value: 0.43943 and parameters: {'alpha': 0.2701394113241932, 'family': 'gaussian', 'lambda': 0.03029229444431384, 'standardize': True}. Best is trial 5 with value: 0.43876544.


██████████████████████████████████████████| (done) 100%
stackedensemble Model Build progress: |

[I 2023-12-17 05:28:48,928] Trial 19 finished with value: 0.45703736 and parameters: {'alpha': 0.12911295576519033, 'family': 'gaussian', 'lambda': 1.020830529008062, 'standardize': False}. Best is trial 5 with value: 0.43876544.


██████████████████████████████████████████| (done) 100%


In [ ]:
#build best ensembler

In [23]:
best_meta_model_params = study.best_params
best_meta_model_params

{'alpha': 0.16165874818526615,
 'family': 'gaussian',
 'lambda': 0.0835053698118708,
 'standardize': False}

In [24]:
best_ensemble = H2OStackedEnsembleEstimator(metalearner_algorithm='glm',
                                            metalearner_params=best_meta_model_params,
                                            metalearner_nfolds=5,
                                            base_models=base_models,
                                            seed=seed)

best_ensemble.train(x=x, y=y, training_frame=train)

stackedensemble Model Build progress: |██████████████████████████████████████████| (done) 100%


Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: StackedEnsemble_model_python_1702787608131_61


Model Summary for Stacked Ensemble: 
key                                        value
-----------------------------------------  ----------------------------------------------------------------------------------------------------------------
Stacking strategy                          cross_validation
Number of base models (used / total)       16/40
# XGBoost base models (used / total)       3/20
# DeepLearning base models (used / total)  13/20
Metalearner algorithm                      GLM
Metalearner fold assignment scheme         Random
Metalearner nfolds                         5
Metalearner fold_column
Custom metalearner hyperparameters         {"alpha": [0.16165874818526615], "family": ["gaussian"], "lambda": [0.0835053698118708], "standardize": [false]}

ModelMetricsRegressionGLM: stackedensemble
** Reported on train data. **

MSE: 0.06267824566736704
RMSE: 0.2503562375243865
MAE: 0.18290489973674476
RMSLE: NaN
Mean Residual Deviance: 0.06267824566736704
R^2: 0.9899834588855161
Null degrees of freedom: 786
Residual degrees of freedom: 770
Null deviance: 4924.63204377904
Residual deviance: 49.327779340217866
AIC: 89.6232018414531

ModelMetricsRegressionGLM: stackedensemble
** Reported on cross-validation data. **

MSE: 0.19585863662750608
RMSE: 0.4425591899706819
MAE: 0.2881158052713775
RMSLE: NaN
Mean Residual Deviance: 0.19585863662750608
R^2: 0.9687000479110793
Null degrees of freedom: 786
Residual degrees of freedom: 771
Null deviance: 4946.3232240102
Residual deviance: 154.14074702584728
AIC: 984.3142615114875

Cross-Validation Metrics Summary: 
                        mean      sd         cv_1_valid    cv_2_valid    cv_3_valid    cv_4_valid    cv_5_valid
----------------------  --------  ---------  ------------  ------------  ------------  ------------  ------------
mae                     0.288018  0.0300084  0.276539      0.294635      0.295134      0.327959      0.245822
mean_residual_deviance  0.196724  0.0838715  0.125475      0.162024      0.173784      0.341868      0.180468
mse                     0.196724  0.0838715  0.125475      0.162024      0.173784      0.341868      0.180468
null_deviance           989.265   54.2427    1020.85       914.544       1058.81       977.375       974.747
r2                      0.968056  0.0145715  0.979606      0.971458      0.974828      0.942659      0.971728
residual_deviance       30.8282   12.854     20.7034       26.0859       26.5889       53.3314       27.4312
rmse                    0.436626  0.087188   0.354225      0.402522      0.416874      0.584695      0.424816
rmsle                   nan       0          nan           nan           nan           nan           nan

[tips]
Use `model.explain()` to inspect the model.
--
Use `h2o.display.toggle_user_tips()` to switch on/off this section.

In [55]:
#see best ensembler performance on val set

In [25]:
ensemble_val_rmse = best_ensemble.model_performance(val).rmse()
ensemble_val_rmse

0.31475634111745304

In [ ]:
#get predictions for test set

In [28]:
test_clean_h2o = h2o.H2OFrame(test_clean)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [30]:
test_clean_h2o['MODE'] = test_clean_h2o['MODE'].asfactor()
test_clean_h2o['POWER'] = test_clean_h2o['POWER'].asfactor()

In [31]:
predictions_h2o = best_ensemble.predict(test_clean_h2o)
predictions = predictions_h2o.as_data_frame()

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


In [32]:
predictions.to_csv('test_preds.csv', header=None, index=False)

In [33]:
predictions

,predict
0,2.511439
1,2.704789
2,2.432484
3,2.336090
4,2.177502
...,...
2495,2.640867
2496,2.618526
2497,2.167043
2498,2.208762
